# Evaluate: Baseline/DL Curves (NB-IoT NPRACH)
این نوت‌بوک خروجی‌های اصلی (FPR/FNR vs CFO, FNR vs SNR, FPR/FNR vs Ptx, NMSE vs SNR) را برای مسیر مبنا و DL تولید می‌کند.

In [ ]:
# پیکربندی خودکار + نخ‌ها + لاگ کمتر
from runtime.auto_config import get_system_profile, recommend_settings, apply_tf_settings, summarize
prof = get_system_profile()
rec  = recommend_settings(prof, mode='eval')
apply_tf_settings(rec)
print(summarize(prof, rec))

import os, numpy as np, tensorflow as tf, matplotlib.pyplot as plt
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
tf.random.set_seed(7)
np.random.seed(7)

%matplotlib inline

os.makedirs('results', exist_ok=True)
def savefig(path):
    plt.savefig(path + '.png', dpi=220, bbox_inches='tight')
    plt.savefig(path + '.pdf', bbox_inches='tight')


In [ ]:
# انتخاب روش و ساخت سیستم
from e2e import E2E
BATCH = 32   # اگر کند بود 8 یا 16 کنید
RUNS  = 6    # برای خروجی سریع 3–4

METHOD = 'baseline'  # یا 'dl'
NPRACH_NUM_SC = 48 if METHOD=='dl' else 24

sys = E2E(METHOD, False, nprach_num_rep=1, nprach_num_sc=NPRACH_NUM_SC, fft_size=256, pfa=0.999)


In [ ]:
# بارگذاری وزن‌ها (فقط برای DL)
import pickle
if METHOD == 'dl':
    T = sys.num_time_samples
    y0 = tf.complex(tf.zeros([1, T], tf.float32), tf.zeros([1, T], tf.float32))
    _  = sys.synch(y0)
    with open('weights.dat','rb') as f:
        sys.synch.set_weights(pickle.load(f))
    print('Weights loaded for DeepNSynch')


In [ ]:
# توابع کمکی: تبدیل CFO و باینینگ SNR
from e2e.cfo import CFO
from parameters import CARRIER_FREQ, SAMPLING_FREQUENCY

cfo_layer = CFO(CARRIER_FREQ, SAMPLING_FREQUENCY)
ppm2norm = lambda ppm: cfo_layer.ppm2Foffnorm(tf.constant(ppm, tf.float32)).numpy()

def bin_by_snr(snr_tensor, metric_tensor, nbins=10):
    import numpy as _np
    snr = snr_tensor.numpy().ravel()
    met = metric_tensor.numpy().ravel()
    used = snr > 0.0
    if not used.any():
        return _np.array([]), _np.array([])
    snr_db = 10.0*_np.log10(snr[used] + 1e-12)
    met_u  = met[used]
    edges = _np.linspace(snr_db.min(), snr_db.max(), nbins+1)
    centers = 0.5*(edges[:-1] + edges[1:])
    idx = _np.digitize(snr_db, edges[1:-1], right=False)
    means = [_np.mean(met_u[idx==b]) if _np.any(idx==b) else _np.nan for b in range(nbins)]
    return centers, _np.array(means)


## Figure A — FPR/FNR در برابر CFO (CFO ثابت)

In [ ]:
import numpy as np, matplotlib.pyplot as plt, tensorflow as tf
targets_ppm  = np.array([0.0, 10.0, 20.0], float)
targets_norm = np.array([ppm2norm(x) for x in targets_ppm], float)
tol_rel, tol_abs0 = 0.1, ppm2norm(0.5)
fpr_mean, fnr_mean, fpr_std, fnr_std = [], [], [], []
for t_ppm, t_norm in zip(targets_ppm, targets_norm):
    vals_fpr, vals_fnr, w_fpr, w_fnr = [], [], [], []
    for _ in range(RUNS):
        snr, toa, f_off, ue_prob, fpr, fnr, toa_err, f_off_err = sys(BATCH, max_cfo_ppm=float(max(targets_ppm)), ue_prob=0.5)
        f_abs = np.abs(f_off.numpy().ravel())
        mask = (f_abs <= tol_abs0) if t_ppm==0.0 else (np.abs(f_abs - abs(t_norm)) <= abs(t_norm)*tol_rel)
        if not mask.any(): continue
        fpr_v = fpr.numpy().ravel()[mask]; fpr_v = fpr_v[fpr_v >= 0.0]
        fnr_v = fnr.numpy().ravel()[mask]; fnr_v = fnr_v[fnr_v >= 0.0]
        if fpr_v.size>0: vals_fpr.append(float(fpr_v.mean())); w_fpr.append(int(fpr_v.size))
        if fnr_v.size>0: vals_fnr.append(float(fnr_v.mean())); w_fnr.append(int(fnr_v.size))
    if vals_fpr:
        w = np.array(w_fpr,float); w/=w.sum(); fpr_mean.append(float(np.sum(w*np.array(vals_fpr))))
        fpr_std.append(float(np.std(vals_fpr, ddof=1)) if len(vals_fpr)>1 else 0.0)
    else: fpr_mean.append(np.nan); fpr_std.append(0.0)
    if vals_fnr:
        w = np.array(w_fnr,float); w/=w.sum(); fnr_mean.append(float(np.sum(w*np.array(vals_fnr))))
        fnr_std.append(float(np.std(vals_fnr, ddof=1)) if len(vals_fnr)>1 else 0.0)
    else: fnr_mean.append(np.nan); fnr_std.append(0.0)
plt.figure(figsize=(6,4))
plt.errorbar(targets_ppm, fpr_mean, yerr=fpr_std, fmt='o-', capsize=3, label='FPR')
plt.errorbar(targets_ppm, fnr_mean, yerr=fnr_std, fmt='s-', capsize=3, label='FNR')
plt.xlabel('CFO (ppm)'); plt.ylabel('Rate'); plt.title(f'{METHOD}: FPR/FNR vs fixed CFO'); plt.grid(True); plt.legend(); plt.tight_layout()
savefig('results/fpr_fnr_vs_cfo_'+METHOD); plt.show()


## Figure B — FNR در برابر SNR (CFO∈{0,10,20} ppm)

In [ ]:
import numpy as np, matplotlib.pyplot as plt, tensorflow as tf
targets_ppm  = [0.0, 10.0, 20.0]
targets_norm = [ppm2norm(x) for x in targets_ppm]
tol_rel, tol_abs0 = 0.1, ppm2norm(0.5)
NBINS=10
plt.figure(figsize=(6,4))
for t_ppm, t_norm in zip(targets_ppm, targets_norm):
    xs_acc, ys_acc = [], []
    for _ in range(RUNS):
        snr, toa, f_off, ue_prob, fpr, fnr, toa_err, f_off_err = sys(BATCH, max_cfo_ppm=float(max(targets_ppm)), ue_prob=0.5)
        f_abs = np.abs(f_off.numpy().ravel())
        mask = (f_abs <= tol_abs0) if t_ppm==0.0 else (np.abs(f_abs - abs(t_norm)) <= abs(t_norm)*tol_rel)
        if not mask.any(): continue
        fnr_sel = tf.convert_to_tensor(fnr.numpy().ravel()[mask], tf.float32)
        fnr_sel = tf.boolean_mask(fnr_sel, tf.greater_equal(fnr_sel, 0.0))
        snr_sel = tf.convert_to_tensor(snr.numpy().ravel()[mask], tf.float32)
        if tf.size(fnr_sel)==0: continue
        centers, means = bin_by_snr(tf.reshape(snr_sel,[-1]), tf.reshape(fnr_sel,[-1]), nbins=NBINS)
        if centers.size>0: xs_acc.append(centers); ys_acc.append(means)
    if xs_acc:
        x0 = xs_acc[0]; y_stack = np.vstack([y for y in ys_acc if y.shape==x0.shape]); y_mean = np.nanmean(y_stack, axis=0)
        plt.plot(x0, y_mean, '-o', label=f'CFO={t_ppm:.0f} ppm')
plt.xlabel('SNR (dB)'); plt.ylabel('FNR (binned mean)')
plt.title(f'{METHOD}: FNR vs SNR at fixed CFO'); plt.grid(True); plt.legend(); plt.tight_layout()
savefig('results/fnr_vs_snr_'+METHOD); plt.show()


## Figure C — FPR/FNR در برابر Ptx (CFO=10 ppm)

In [ ]:
import numpy as np, matplotlib.pyplot as plt, tensorflow as tf
PTX = np.array([0.2, 0.5, 0.8], float)
t_ppm = 10.0; t_norm = float(ppm2norm(t_ppm)); tol_rel = 0.1
fpr_m, fnr_m = [], []
for p in PTX:
    vals_fpr, vals_fnr, w_fpr, w_fnr = [], [], [], []
    for _ in range(RUNS):
        snr, toa, f_off, ue_prob, fpr, fnr, toa_err, f_off_err = sys(BATCH, max_cfo_ppm=t_ppm, ue_prob=float(p))
        f_abs = np.abs(f_off.numpy().ravel()); mask = np.abs(f_abs - abs(t_norm)) <= abs(t_norm)*tol_rel
        if not mask.any(): continue
        fpr_v = fpr.numpy().ravel()[mask]; fpr_v = fpr_v[fpr_v >= 0.0]
        fnr_v = fnr.numpy().ravel()[mask]; fnr_v = fnr_v[fnr_v >= 0.0]
        if fpr_v.size>0: vals_fpr.append(float(fpr_v.mean())); w_fpr.append(int(fpr_v.size))
        if fnr_v.size>0: vals_fnr.append(float(fnr_v.mean())); w_fnr.append(int(fnr_v.size))
    if vals_fpr: w=np.array(w_fpr,float); w/=w.sum(); fpr_m.append(float(np.sum(w*np.array(vals_fpr))))
    else: fpr_m.append(np.nan)
    if vals_fnr: w=np.array(w_fnr,float); w/=w.sum(); fnr_m.append(float(np.sum(w*np.array(vals_fnr))))
    else: fnr_m.append(np.nan)
plt.figure(figsize=(6,4))
plt.plot(PTX, fpr_m, 'o-', label='FPR'); plt.plot(PTX, fnr_m, 's-', label='FNR')
plt.xlabel('Ptx'); plt.ylabel('Rate'); plt.title(f'{METHOD}: FPR/FNR vs Ptx @ CFO=10 ppm'); plt.grid(True); plt.legend(); plt.tight_layout()
savefig('results/fpr_fnr_vs_ptx_'+METHOD); plt.show()


## Figure D — NMSE مربوط به ToA و CFO در برابر SNR

In [ ]:
import numpy as np, matplotlib.pyplot as plt
NBINS=10
xs_acc_t, ys_acc_t, xs_acc_f, ys_acc_f = [], [], [], []
for _ in range(RUNS):
    snr, toa, f_off, ue_prob, fpr, fnr, toa_err, f_off_err = sys(BATCH, max_cfo_ppm=10.0, ue_prob=0.5)
    def bin_mean(x, y, nbins=10):
        x = x.ravel(); y = y.ravel()
        used = x > 0.0
        x = x[used]; y = y[used]
        if x.size == 0: return np.array([]), np.array([])
        x_db = 10.0*np.log10(x + 1e-12)
        edges = np.linspace(x_db.min(), x_db.max(), nbins+1)
        centers = 0.5*(edges[:-1] + edges[1:])
        idx = np.digitize(x_db, edges[1:-1], right=False)
        means = [np.mean(y[idx==b]) if np.any(idx==b) else np.nan for b in range(nbins)]
        return centers, np.array(means)
    c_t, m_t = bin_mean(snr.numpy(), toa_err.numpy(), nbins=NBINS)
    c_f, m_f = bin_mean(snr.numpy(), f_off_err.numpy(), nbins=NBINS)
    if c_t.size>0: xs_acc_t.append(c_t); ys_acc_t.append(m_t)
    if c_f.size>0: xs_acc_f.append(c_f); ys_acc_f.append(m_f)

def avg_curves(xs, ys):
    if not xs: return np.array([]), np.array([])
    x0 = xs[0]; y_stack = np.vstack([y for y in ys if y.shape==x0.shape])
    return x0, np.nanmean(y_stack, axis=0)

x_t, y_t = avg_curves(xs_acc_t, ys_acc_t)
x_f, y_f = avg_curves(xs_acc_f, ys_acc_f)

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
if x_t.size>0: plt.plot(x_t, y_t, '-o')
plt.title(f'{METHOD}: ToA NMSE vs SNR'); plt.xlabel('SNR (dB)'); plt.ylabel('NMSE'); plt.grid(True)
plt.subplot(1,2,2)
if x_f.size>0: plt.plot(x_f, y_f, '-o')
plt.title(f'{METHOD}: CFO NMSE vs SNR'); plt.xlabel('SNR (dB)'); plt.ylabel('NMSE'); plt.grid(True)
plt.tight_layout()
savefig('results/nmse_vs_snr_'+METHOD); plt.show()
